In [23]:
%load_ext autoreload
%autoreload 2

import pandas as pd

# Set display options to show more columns and wider output
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Don't truncate column contents

from IPython.display import display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Tasks
## Backlog
- Remove "Using closest match for encoding: 'gpt-4o' for input model 'gpt-4o-2024-08-06'"
- Remove \n signs
- Create organizations list with translation to he and en
## Done
- Pass for each event the last timestemp an update apeared
- Pass the OpenAI client json that is loaded with data the the promot will use
- Add total cost for OpenAI client

# אפיון בסיסי
- להתריע על אירועים לפי גזרה מוגדרת שמעניינת אותנו
- לא כל אירוע שמתקיים בארץ ואפילו בצפון מעניין אותנו
- צריך להגדיר בצורה מדויקת מה מעניין ומה לא

# Read Messeges

In [8]:
import json
import pandas as pd

# Specify the file path
file_path = "telegram_messages.json"  # Replace with the actual file path if different

# Read the JSON file
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert the JSON data to a Pandas DataFrame
df = pd.json_normalize(data)

# Display the DataFrame
df.head()

,channel,message_id,timestamp,message,media,metadata.sender_id,metadata.message_type
0,From_hebron,270263,2024-10-13 19:32:55,♦️إعلام العدو يتحدث عن حدث أمني صعب.,[],-1001478404564,Message
1,From_hebron,270262,2024-10-13 19:32:20,♦️ يسرائيل للو تسنزورا: إصابة قاعدة تدريب غولا...,[],-1001478404564,Message
2,From_hebron,270261,2024-10-13 19:31:11,محكمة الاحتلال تُمدد الحكم على الأسير سليم الر...,"[{'media_type': 'MessageMediaPhoto', 'media_id...",-1001478404564,Message
3,From_hebron,270260,2024-10-13 19:29:24,♦️المتحدثة باسم الشرطة الصهيونيه : المتظاهرون ...,"[{'media_type': 'MessageMediaDocument', 'media...",-1001478404564,Message
4,From_hebron,270259,2024-10-13 19:29:07,♦️ القسام يعلن تفجير عين نفق في قوة اسرائيلية ...,[],-1001478404564,Message


# Translate Messages

In [73]:
import pandas as pd
from typing import Optional, Dict, List
from pydantic import BaseModel
from openai_client import OpenAIClient
from tqdm import tqdm

class TranslationResponse(BaseModel):
    """Response model for translations."""
    hebrew: str
    english: str

class NERResponse(BaseModel):
    """Response model for Named Entity Recognition (NER)."""
    locations: List[str]
    people: List[str]
    organizations: List[str]

class MessageTranslator:
    """Processor class to handle Telegram message translation."""
    
    def __init__(self, client: OpenAIClient):
        self.client = client

    def translate_message(self, message: str) -> Optional[TranslationResponse]:
        """Translate a Telegram message into both Hebrew and English."""
        system_message = """
            Act as a highly accurate translator. Your task is to take an Arabic message from a Telegram group and translate it into both 
            Hebrew and English.

            Ensure that:
            
            - All emojis and decorative symbols (such as bullets, arrows, or icons) are removed from the original message before translation.
            - The translations are precise and maintain the original meaning, tone, and nuance, so the user fully understands both the context 
              and sentiment.
            - Military, organizational, or specific terminology (e.g., names of groups, military actions) should be translated consistently 
              without alteration or substitution.
            - Prioritize clarity and natural phrasing in both Hebrew and English, avoiding overly literal translations, while maintaining 
              fidelity to the original message.
            - Retain punctuation and formatting (such as parentheses) accurately to ensure readability in both languages.
            - Maintain urgency and emotional tone (e.g., in reports of military strikes, martyrdom) in a way that sounds natural and fluent.
        """
        
        response = self.client.chat(
            system_message=system_message,
            user_message=message,
            response_format=TranslationResponse
        )

        return response

class NERProcessor:
    """Processor class to handle Named Entity Recognition (NER) on Telegram messages."""
    
    def __init__(self, client: OpenAIClient):
        self.client = client

    def extract_entities(self, message: str) -> Optional[NERResponse]:
        """Perform NER to extract locations, people, and organizations from the message."""
        system_message = """
            Act as an entity recognition system. Your task is to extract the following entities from the provided Arabic Telegram message:
            
            - Locations: Geographical places mentioned in the message.
            - People: Names of individuals mentioned.
            - Organizations: Names of groups, companies, or military organizations mentioned.

            Ensure that:
            - You return only the names without extra context or symbols.
            - If no entities are found, return empty lists for each category.
        """
        
        response = self.client.chat(
            system_message=system_message,
            user_message=message,
            response_format=NERResponse
        )

        return response

class MainProcessor:
    """Class to manage the full pipeline: reading messages, translating, extracting entities, and updating DataFrame."""
    
    def __init__(self, translator: MessageTranslator, ner_processor: NERProcessor, df: pd.DataFrame):
        self.translator = translator
        self.ner_processor = ner_processor
        self.df = df.copy()  # Avoid modifying the original df

        # Prepare new columns for translations and entity recognition
        self.df['hebrew_translation'] = None
        self.df['english_translation'] = None
        self.df['locations'] = None
        self.df['people'] = None
        self.df['organizations'] = None

    def run(self) -> pd.DataFrame:
        """Run the full pipeline, translating each message, extracting entities, and returning an updated DataFrame."""
        for index, row in tqdm(self.df.iterrows(), total=len(self.df), desc="Processing messages"):
            message = row['message']
            
            # Perform translation
            translation = self.translator.translate_message(message)
            if translation:
                self.df.at[index, 'hebrew_translation'] = translation.hebrew
                self.df.at[index, 'english_translation'] = translation.english
            else:
                print(f"No translation found for message_id {row['message_id']}.")

            # Perform NER
            ner_data = self.ner_processor.extract_entities(translation.english)
            if ner_data:
                self.df.at[index, 'locations'] = ", ".join(ner_data.locations)
                self.df.at[index, 'people'] = ", ".join(ner_data.people)
                self.df.at[index, 'organizations'] = ", ".join(ner_data.organizations)
            else:
                print(f"No NER data found for message_id {row['message_id']}.")

        # Return the updated DataFrame with translations and NER data
        return self.df

if __name__ == "__main__":
    # Specify the file path
    file_path = "telegram_messages.json"  # Replace with the actual file path if different
    
    # Read the JSON file
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    # Convert the JSON data to a Pandas DataFrame
    df = pd.json_normalize(data)

    # Initialize the OpenAI client, translators, and processors
    openai_client = OpenAIClient()
    message_translator = MessageTranslator(openai_client)
    ner_processor = NERProcessor(openai_client)
    main_processor = MainProcessor(message_translator, ner_processor, df)

    # Run the pipeline and get the updated DataFrame
    updated_df = main_processor.run()

    # Print the total costs of OpenAI calls
    openai_client.print_total_costs()

    # Display the updated DataFrame with translations and entity recognition
    display(updated_df[['message', 'hebrew_translation', 'english_translation', 'locations', 'people', 'organizations']])


Using closest match for encoding: 'gpt-4o' for input model 'gpt-4o-2024-08-06'
Resolved model for encoding: 'gpt-4o'
Using pricing model: 'gpt-4o-2024-08-06'


Processing messages: 100%|███████████████████████████████████████| 25/25 [01:15<00:00,  3.03s/it]

Total prompt cost: $0.021985
Total completion cost: $0.020620
Overall total cost: $0.042605


,message,hebrew_translation,english_translation,locations,people,organizations
0,♦️إعلام العدو يتحدث عن حدث أمني صعب.,תקשורת האויב מדברת על אירוע ביטחוני קשה.,Enemy media talks about a serious security incident.,,,
1,♦️ يسرائيل للو تسنزورا: إصابة قاعدة تدريب غولاني \nبطائرة\n مسيرة تابعة للحزب في عكا قبل قليل,ישראל לא תצנזור: פגיעה בבסיס אימונים של גולני ברחפן של החזבאללה בעכו לפני זמן קצר.,Israel will not censor: Golani training base hit by a Hezbollah drone in Acre recently.,Acre,,"Israel, Golani, Hezbollah"
2,محكمة الاحتلال تُمدد الحكم على الأسير سليم الرجوب من سكان حي كريسة بمدينة دورا للمرة الثالثة على التوالي لمدة ستة أشهر إضافية.,"בית המשפט של הכיבוש מאריך את פסק הדין על האסיר סלים רג'וב, תושב שכונת קריסה בעיר דורה, בפעם השלישית ברציפות למשך שישה חודשים נוספים.","The occupation court extends the sentence of detainee Salim Rajoub, a resident of the Karisah neighborhood in the city of Dura, for the third consecutive time for an additional six months.","Karisah, Dura",Salim Rajoub,
3,♦️المتحدثة باسم الشرطة الصهيونيه : المتظاهرون ضد الحكومة عرضوا حياة ضباط الشرطة للخطر أمس عندما تعمدوا إشعال حريق كبير على بعد بوصات منهم.,דוברת המשטרה הציונית: המפגינים נגד הממשלה סיכנו את חיי קציני המשטרה אתמול כאשר בכוונה תחילה הציתו שריפה גדולה במרחק של אינצ'ים מהם.,The spokesperson for the Zionist police: The demonstrators against the government endangered the lives of police officers yesterday when they deliberately ignited a large fire just inches away from them.,,,Zionist police
4,♦️ القسام يعلن تفجير عين نفق في قوة اسرائيلية راجلة شمال مدينة بيت لاهيا شمال القطاع,הקסאם מכריז על פיצוץ פתח מנהרה בכוח ישראלי רגלי בצפון העיר בית להיה בצפון הרצועה,Al-Qassam announces the detonation of a tunnel entrance against an Israeli foot patrol north of Beit Lahia in the northern part of the Gaza Strip,"Beit Lahia, Gaza Strip",,Al-Qassam
5,🛑اعتراض الطائرة بدون طيار في سماء كريات قبل قليل,יירוט כלי טיס בלתי מאויש מעל קריית לפני זמן קצר,Interception of an unmanned aerial vehicle over Kiryat a short while ago,Kiryat,,
6,"🚫🟠🟠 🟢 كتائب القسام تستهدف ناقلة جند صهيونية بقذيفة ""تاندوم"" شمال مدينة بيت لاهيا شمال القطاع","גדודי עז א-דין אל-קסאם תוקפים רכב משוריין ישראלי עם רקטה ""טאנדם"" בצפון בית לייה בצפון הרצועה",Al-Qassam Brigades target a Zionist armored vehicle with a 'Tandem' missile north of Beit Lahia in the northern Gaza Strip,"Beit Lahia, northern Gaza Strip",,Al-Qassam Brigades
7,♦️بالفيديو | كلمة سماحة الأمين العام الشهيد الي ( المجاهدين خلال إحدى المناورات العسكرية,"בסרטון | נאום של מזכ""ל הקדוש, השהיד, לאחים הלוחמים במהלך אחד התרגילים הצבאיים","In the video | Speech by the Sacred Secretary-General, the martyr, to the mujahideen during one of the military exercises",,,
8,♦️عاجل| حزب الله يشن هجوماً مركباً بالطائرات المسيّرة والصواريخ على شمال فلسطين المحتلة.,דחוף| חזבאללה מבצע התקפה משולבת באמצעות רחפנים וטילים על צפון פלסטין הכבושה.,Urgent| Hezbollah launches a combined attack with drones and missiles on northern occupied Palestine.,northern occupied Palestine,,Hezbollah
9,♦️التقارير الأولية دون رقابة: \nإصابة مستوطن نتيجة سقوط شظايا في منطقة كريات شمونة.,דיווחים ראשוניים ללא צנזורה: נפגע מתנחל כתוצאה מנפילת רסיסים באזור קריית שמונה.,Initial uncensored reports: A settler was injured due to falling shrapnel in the Kiryat Shmona area.,Kiryat Shmona,,


In [57]:
import pandas as pd
from datetime import datetime, timedelta
from pydantic import BaseModel
from typing import List, Optional

class EventUpdateResponse(BaseModel):
    """Response model for event updates."""
    event_type: str
    summary: str
    event_name: Optional[str] = None  # Optional argument to handle event naming

class TelegramMessageProcessor:
    """Processor class to handle Telegram message classification and updates."""
    
    def __init__(self, client: OpenAIClient):
        self.client = client
        self.active_events = []  # List to track currently active events
    
    def classify_message(self, message: str, current_events: List[str]) -> Optional[EventUpdateResponse]:
        """Classify a Telegram message into an event type and generate a summary."""
        system_message = f"""
            You are an event classifier. Classify the following message into one of the event types: Protest, Fire, Accident, or Unknown.

            After classification, summarize the message briefly and suggest if this message belongs to any of the following ongoing events: {', '.join(current_events)}.
            If the message belongs to a new event, create a new event name.

            Event Types:
            - Protest
            - Fire
            - Accident
            - Unknown
            
            Message:
        """
        
        response = self.client.chat(
            system_message=system_message,
            user_message=message,
            response_format=EventUpdateResponse
        )

        if response and response.choices and hasattr(response.choices[0].message, 'parsed'):
            return response.choices[0].message.parsed  # Parsed classification and summary
        return None
    
    def update_event(self, classification: EventUpdateResponse, message: str, timestamp: datetime) -> str:
        """Update an existing event with new information, or create a new event."""
        event_exists = False
        event_name = classification.event_name or f"{classification.event_type} Event {len(self.active_events) + 1}"
        
        for event in self.active_events:
            if event['name'] == event_name:
                # Update existing event
                event['messages'].append(message)
                event['timestamps'].append(timestamp)
                event['last_update'] = timestamp
                event['summary'] = self.summarize_event(event['messages'])
                event_exists = True
                print(f"Message added to existing event: {event_name}")
                break
        
        if not event_exists:
            # Create a new event
            new_event = {
                'type': classification.event_type,
                'name': event_name,
                'messages': [message],
                'timestamps': [timestamp],
                'first_report': timestamp,
                'last_update': timestamp,
                'summary': classification.summary  # Initial summary based on the classification
            }
            self.active_events.append(new_event)
            print(f"New event created: {event_name}")
        
        print(f"Message: {message}, Event Type: {classification.event_type}, Summary: {classification.summary}")
        return event_name
    
    def summarize_event(self, messages: List[str]) -> str:
        """Summarize the event by combining related messages."""
        system_message = """
            Summarize the following set of messages into a concise and clear event summary:
        """
        message_list = "\n".join(messages)
        response = self.client.chat(
            system_message=system_message,
            user_message=message_list
        )
        
        return response.choices[0].message.content if response else "Summary unavailable."

    def process_new_message(self, message: str, timestamp: datetime) -> str:
        """Process a new incoming Telegram message."""
        current_event_names = [event['name'] for event in self.active_events]
        classification = self.classify_message(message, current_event_names)
        
        if classification:
            event_name = self.update_event(classification, message, timestamp)
            return event_name
        else:
            print("Message could not be classified.")
            return "Unclassified"

    def get_active_events(self):
        """Get the current list of active events with summaries."""
        return [{"name": event['name'], "type": event['type'], "summary": event['summary'], "last_update": event['last_update']} for event in self.active_events]

    def get_event_timeline(self, event_name: str):
        """Get the timeline of updates for a specific event."""
        for event in self.active_events:
            if event['name'] == event_name:
                print(f"Timeline for {event_name}:")
                for msg, time in zip(event['messages'], event['timestamps']):
                    print(f"{time}: {msg}")
                return
        print(f"No event found with the name {event_name}.")


# Define static timestamps for 25 messages
timestamps = [
    datetime(2024, 10, 12, 8, 0), datetime(2024, 10, 12, 8, 5), datetime(2024, 10, 12, 8, 10),
    datetime(2024, 10, 12, 8, 20), datetime(2024, 10, 12, 8, 25), datetime(2024, 10, 12, 8, 30),
    datetime(2024, 10, 12, 8, 40), datetime(2024, 10, 12, 8, 45), datetime(2024, 10, 12, 8, 50),
    datetime(2024, 10, 12, 9, 0), datetime(2024, 10, 12, 9, 5), datetime(2024, 10, 12, 9, 10),
    datetime(2024, 10, 12, 9, 15), datetime(2024, 10, 12, 9, 20), datetime(2024, 10, 12, 9, 25),
    datetime(2024, 10, 12, 9, 30), datetime(2024, 10, 12, 9, 35), datetime(2024, 10, 12, 9, 40),
    datetime(2024, 10, 12, 9, 45), datetime(2024, 10, 12, 9, 50), datetime(2024, 10, 12, 10, 0),
    datetime(2024, 10, 12, 10, 5), datetime(2024, 10, 12, 10, 10), datetime(2024, 10, 12, 10, 15),
    datetime(2024, 10, 12, 10, 20)
]

# 25 Fake messages
fake_messages = [
    "Protest in London against tax reforms.",
    "Fire breaks out near the protest area in London.",
    "More protesters joining the march in Manchester.",
    "Accident on the highway blocks traffic in Birmingham.",
    "Massive protests erupt in downtown Paris.",
    "A large fire reported in an industrial complex.",
    "Protesters clash with police in Madrid.",
    "Earthquake detected near the city of Istanbul.",
    "More people join the protest in Barcelona.",
    "Firefighters are battling a fire in a residential area.",
    "Protest in New York over climate policies.",
    "Explosion heard near a protest in Beirut.",
    "Flooding reported after heavy rains in Venice.",
    "Protests in Hong Kong demanding political reforms.",
    "Accident involving multiple cars reported in Berlin.",
    "Large protests continue in downtown Paris.",
    "The fire in London is now under control.",
    "Accident cleared on Birmingham highway.",
    "Additional protests erupt in Rome.",
    "Major fire reported in Los Angeles.",
    "More casualties reported in the Beirut explosion.",
    "Traffic accident blocks main road in Tokyo.",
    "Earthquake aftershocks detected in Istanbul.",
    "Protest in Washington, D.C., over climate policies.",
    "Fire spreads to nearby buildings in Los Angeles."
]

# Create a pandas DataFrame to store messages, timestamps, and event names
df = pd.DataFrame({
    "message": fake_messages,
    "timestamp": timestamps
})

# Simulating the processing of the messages
client = OpenAIClient(show_prices=False)  # Initialize OpenAI client
processor = TelegramMessageProcessor(client)

# Process each fake message and store the event name in the DataFrame
for index, row in df.iterrows():
    message = row["message"]
    timestamp = row["timestamp"]
    event_name = processor.process_new_message(message, timestamp)
    df.at[index, "event_name"] = event_name

# Check active events after processing
active_events = processor.get_active_events()
for event in active_events:
    print(f"Event Name: {event['name']}, Event Type: {event['type']}, Last Update: {event['last_update']}, Summary: {event['summary']}\n")

# Display the DataFrame (with messages, timestamps, and event names)
print(df)

# Get the timeline for a specific event (e.g., "Protest Event 1")
processor.get_event_timeline("Protest Event 1")

Using closest match for encoding: 'gpt-4o' for input model 'gpt-4o-2024-08-06'
Resolved model for encoding: 'gpt-4o'
Using pricing model: 'gpt-4o-2024-08-06'
New event created: London Tax Reform Protest
Message: Protest in London against tax reforms., Event Type: Protest, Summary: Protest in London specifically addressing tax reforms.
New event created: London Protest Area Fire
Message: Fire breaks out near the protest area in London., Event Type: Fire, Summary: A fire has erupted close to the area where a protest is taking place in London.
New event created: Manchester March Protest
Message: More protesters joining the march in Manchester., Event Type: Protest, Summary: More protesters are joining a march in Manchester.
New event created: Birmingham Highway Accident
Message: Accident on the highway blocks traffic in Birmingham., Event Type: Accident, Summary: An accident on the highway in Birmingham has caused a traffic blockage.
New event created: Paris Downtown Protest
Message: Mass

In [62]:
display(df[['event_name', 'message', 'timestamp']].sort_values(by=['event_name', 'timestamp'], ascending=[True, True]))
print(df['event_name'].value_counts())

,event_name,message,timestamp
13,Accident in Berlin Involving Multiple Cars,Accident involving multiple cars reported in Berlin.,2024-10-12 09:05:00
3,Accident on Birmingham Highway,Accident on the highway blocks traffic in Birmingham.,2024-10-12 08:15:00
17,Accident on Birmingham Highway,The highway accident in Birmingham has been cleared.,2024-10-12 09:25:00
7,Earthquake Near Istanbul,Earthquake detected near the city of Istanbul.,2024-10-12 08:35:00
20,Earthquake Near Istanbul,Aftershocks of the earthquake felt in Istanbul.,2024-10-12 09:40:00
10,Explosion Near Beirut Protest,Explosion heard near a protest in Beirut.,2024-10-12 08:50:00
24,Explosion Near Beirut Protest,Beirut protestors clash with security forces.,2024-10-12 10:00:00
1,Fire Incident Near London Protest,Fire breaks out near the protest area in London.,2024-10-12 08:05:00
15,Fire Incident Near London Protest,The fire in London is now under control.,2024-10-12 09:15:00
5,Fire at Industrial Complex,A large fire reported in an industrial complex.,2024-10-12 08:25:00


event_name
New York Protest Over Climate Policies        3
Fire Incident Near London Protest             2
Accident on Birmingham Highway                2
Paris Protest                                 2
Fire at Industrial Complex                    2
Madrid Protest and Police Clash               2
Earthquake Near Istanbul                      2
Fire in Residential Area                      2
Explosion Near Beirut Protest                 2
London Protest Against Tax Reforms            1
Manchester Protest                            1
Venice Flooding After Heavy Rains             1
Hong Kong Protest for Political Reforms       1
Accident in Berlin Involving Multiple Cars    1
Rome Protest                                  1
Name: count, dtype: int64


In [59]:
# After processing the events, we want to prepare the final DataFrame including the original messages
final_event_data = []

for event in processor.active_events:
    # For each event, we will add each original message and its summary as separate rows
    for msg, timestamp in zip(event['messages'], event['timestamps']):
        final_event_data.append({
            'event_name': event['name'],
            'message': msg,
            'summary': event['summary'],
            'timestamp': timestamp
        })

# Convert the event data to a DataFrame
final_df = pd.DataFrame(final_event_data)

# Sort the DataFrame by event name and timestamp for better readability
final_df = final_df.sort_values(by=['event_name', 'timestamp'])

# Display the final DataFrame (messages, summaries, timestamps, and event names)
final_df[['event_name', 'message', 'summary', 'timestamp']].sort_values(by=['event_name', 'timestamp'], ascending=[True, True])

,event_name,message,summary,timestamp
12,Barcelona Growing Protest,More people join the protest in Barcelona.,The protest in Barcelona is seeing an increase in participation.,2024-10-12 08:50:00
15,Beirut Protest Explosion,Explosion heard near a protest in Beirut.,"An explosion occurred near a protest in Beirut, resulting in multiple casualties.",2024-10-12 09:10:00
16,Beirut Protest Explosion,More casualties reported in the Beirut explosion.,"An explosion occurred near a protest in Beirut, resulting in multiple casualties.",2024-10-12 10:00:00
19,Berlin Multi-Car Accident,Accident involving multiple cars reported in Berlin.,A multiple car accident has occurred in Berlin.,2024-10-12 09:25:00
4,Birmingham Highway Accident,Accident on the highway blocks traffic in Birmingham.,A highway accident in Birmingham initially caused a traffic blockage but has since been cleared.,2024-10-12 08:20:00
5,Birmingham Highway Accident,Accident cleared on Birmingham highway.,A highway accident in Birmingham initially caused a traffic blockage but has since been cleared.,2024-10-12 09:40:00
18,Hong Kong Political Reform Protest,Protests in Hong Kong demanding political reforms.,There are ongoing protests in Hong Kong where people are demanding political reforms.,2024-10-12 09:20:00
8,Industrial Complex Fire,A large fire reported in an industrial complex.,A large fire has broken out in an industrial complex.,2024-10-12 08:30:00
10,Istanbul Earthquake,Earthquake detected near the city of Istanbul.,"An earthquake occurred near the city of Istanbul, followed by aftershocks in the area.",2024-10-12 08:45:00
11,Istanbul Earthquake,Earthquake aftershocks detected in Istanbul.,"An earthquake occurred near the city of Istanbul, followed by aftershocks in the area.",2024-10-12 10:10:00


- להעביר את המועד של ההודעה הנוכחית ולהגיד למול להחליט אם מדובר על אותו אירוע או לא

בהמשך
- לזהות אם יש לנו אירועים כפולים ולאחד בינם / לעדכן את השמות של האירועים

In [63]:
import pandas as pd
from datetime import datetime, timedelta
from pydantic import BaseModel
from typing import List, Optional

class EventUpdateResponse(BaseModel):
    """Response model for event updates."""
    event_type: str
    event_name: Optional[str] = None  # Optional argument for a new or existing event

class TelegramMessageProcessor:
    """Processor class to handle Telegram message classification and updates."""
    
    def __init__(self, client: OpenAIClient):
        self.client = client
        self.active_events = []  # List to track currently active events

    def classify_message(self, message: str, current_events: List[dict]) -> Optional[EventUpdateResponse]:
        """Classify a Telegram message into an event type and generate an event name."""
        
        # Create a formatted string to pass the current event names and last update timestamps
        event_info = "\n".join([f"Event: {event['name']} (Last update: {event['last_update']})" for event in current_events])
        
        system_message = f"""
            You are an event classifier. Based on the following ongoing events and their last update dates, classify the new message into one of these events or suggest a new event name.

            Ongoing events:
            {event_info}

            Important Note: If the event mentioned in the message seems to refer to something that happened after a long time or a separate instance of a similar event, classify it as a new event.
            
            Message:
        """
        
        response = self.client.chat(
            system_message=system_message,
            user_message=message,
            response_format=EventUpdateResponse
        )

        if response and response.choices and hasattr(response.choices[0].message, 'parsed'):
            return response.choices[0].message.parsed  # Parsed classification with event name
        return None
    
    def update_event(self, classification: EventUpdateResponse, message: str, timestamp: datetime) -> str:
        """Update an existing event with new information, or create a new event."""
        event_name = classification.event_name or f"{classification.event_type} Event {len(self.active_events) + 1}"
        event_exists = False

        for event in self.active_events:
            # Check if the message belongs to this event by name
            if event['name'] == event_name:
                # Update existing event
                event['messages'].append(message)
                event['timestamps'].append(timestamp)
                event['last_update'] = timestamp
                event_exists = True
                print(f"Message added to existing event: {event_name}\n")
                break

        if not event_exists:
            # Create a new event if not found
            new_event = {
                'type': classification.event_type,
                'name': event_name,
                'messages': [message],
                'timestamps': [timestamp],
                'first_report': timestamp,
                'last_update': timestamp
            }
            self.active_events.append(new_event)
            print(f"New event created: {event_name}")

        print(f"Message: {message}, Event Type: {classification.event_type}\n")
        return event_name
    
    def process_new_message(self, message: str, timestamp: datetime) -> str:
        """Process a new incoming Telegram message."""
        current_event_names = [{'name': event['name'], 'last_update': event['last_update']} for event in self.active_events]
        classification = self.classify_message(message, current_event_names)
        
        if classification:
            event_name = self.update_event(classification, message, timestamp)
            return event_name
        else:
            print("Message could not be classified.")
            return "Unclassified"

    def get_active_events(self):
        """Get the current list of active events."""
        return [{"name": event['name'], "type": event['type'], "last_update": event['last_update']} for event in self.active_events]

# Define static timestamps for 25 messages (representing various events, some similar but occurring at different times)
timestamps = [
    datetime(2024, 10, 12, 8, 0), datetime(2024, 10, 12, 8, 5), datetime(2024, 10, 12, 8, 10),
    datetime(2024, 10, 12, 8, 15), datetime(2024, 10, 12, 8, 20), datetime(2024, 10, 12, 8, 25),
    datetime(2024, 10, 12, 8, 30), datetime(2024, 10, 12, 8, 35), datetime(2024, 10, 12, 8, 40),
    datetime(2024, 10, 12, 8, 45), datetime(2024, 10, 12, 8, 50), datetime(2024, 10, 12, 8, 55),
    datetime(2024, 10, 12, 9, 0), datetime(2024, 10, 12, 9, 5), datetime(2024, 10, 12, 9, 10),
    datetime(2024, 10, 12, 9, 15), datetime(2024, 10, 12, 9, 20), datetime(2024, 10, 12, 9, 25),
    datetime(2024, 10, 12, 9, 30), datetime(2024, 10, 12, 9, 35), datetime(2024, 10, 12, 9, 40),
    datetime(2024, 10, 12, 9, 45), datetime(2024, 10, 12, 9, 50), datetime(2024, 10, 12, 9, 55),
    datetime(2024, 10, 12, 10, 0)
]

# 25 Fake messages representing roughly 5 distinct events
fake_messages = [
    "Protest in London against tax reforms.",
    "Fire breaks out near the protest area in London.",
    "More protesters joining the march in Manchester.",
    "Accident on the highway blocks traffic in Birmingham.",
    "Massive protests erupt in downtown Paris.",
    "A large fire reported in an industrial complex.",
    "Protesters clash with police in Madrid.",
    "Earthquake detected near the city of Istanbul.",
    "Firefighters are battling a fire in a residential area.",
    "Protest in New York over climate policies.",
    "Explosion heard near a protest in Beirut.",
    "Flooding reported after heavy rains in Venice.",
    "Protests in Hong Kong demanding political reforms.",
    "Accident involving multiple cars reported in Berlin.",
    "Additional protests erupt in Rome.",
    "The fire in London is now under control.",
    "More protesters join the Paris movement.",
    "The highway accident in Birmingham has been cleared.",
    "The industrial complex fire is spreading to nearby areas.",
    "Protesters in Washington march against climate policies.",
    "Aftershocks of the earthquake felt in Istanbul.",
    "The residential area fire has been put out.",
    "The police in Madrid are trying to control protesters.",
    "Climate activists in New York stage another demonstration.",
    "Beirut protestors clash with security forces."
]

# Create a pandas DataFrame to store messages, timestamps, and event names
df = pd.DataFrame({
    "message": fake_messages,
    "timestamp": timestamps
})

# Simulating the processing of the messages
client = OpenAIClient(show_prices=False)  # Initialize OpenAI client
processor = TelegramMessageProcessor(client)

# Process each fake message and store the event name in the DataFrame
for index, row in df.iterrows():
    message = row["message"]
    timestamp = row["timestamp"]
    event_name = processor.process_new_message(message, timestamp)
    df.at[index, "event_name"] = event_name

# Display the final DataFrame with event names, messages, and timestamps
df[['event_name', 'message', 'timestamp']]

Using closest match for encoding: 'gpt-4o' for input model 'gpt-4o-2024-08-06'
Resolved model for encoding: 'gpt-4o'
Using pricing model: 'gpt-4o-2024-08-06'
New event created: London Tax Reform Protests
Message: Protest in London against tax reforms., Event Type: new

Message added to existing event: London Tax Reform Protests

Message: Fire breaks out near the protest area in London., Event Type: Ongoing Event

New event created: Manchester Tax Reform Protests
Message: More protesters joining the march in Manchester., Event Type: new

New event created: Birmingham Highway Accident
Message: Accident on the highway blocks traffic in Birmingham., Event Type: New Event

New event created: Paris Protests
Message: Massive protests erupt in downtown Paris., Event Type: New Event

New event created: Industrial Complex Fire Incident
Message: A large fire reported in an industrial complex., Event Type: New Event

New event created: Madrid Protests
Message: Protesters clash with police in Madri

,event_name,message,timestamp
0,London Tax Reform Protests,Protest in London against tax reforms.,2024-10-12 08:00:00
1,London Tax Reform Protests,Fire breaks out near the protest area in London.,2024-10-12 08:05:00
2,Manchester Tax Reform Protests,More protesters joining the march in Manchester.,2024-10-12 08:10:00
3,Birmingham Highway Accident,Accident on the highway blocks traffic in Birmingham.,2024-10-12 08:15:00
4,Paris Protests,Massive protests erupt in downtown Paris.,2024-10-12 08:20:00
5,Industrial Complex Fire Incident,A large fire reported in an industrial complex.,2024-10-12 08:25:00
6,Madrid Protests,Protesters clash with police in Madrid.,2024-10-12 08:30:00
7,Istanbul Earthquake,Earthquake detected near the city of Istanbul.,2024-10-12 08:35:00
8,Residential Area Fire Incident,Firefighters are battling a fire in a residential area.,2024-10-12 08:40:00
9,New York Climate Policies Protest,Protest in New York over climate policies.,2024-10-12 08:45:00


In [66]:
display(df[['event_name', 'message', 'timestamp']].sort_values(by=['event_name', 'timestamp'], ascending=[True, True]))
df['event_name'].value_counts()

,event_name,message,timestamp
10,Beirut Protest Explosion,Explosion heard near a protest in Beirut.,2024-10-12 08:50:00
24,Beirut Protest Explosion,Beirut protestors clash with security forces.,2024-10-12 10:00:00
13,Berlin Car Accident Incident,Accident involving multiple cars reported in Berlin.,2024-10-12 09:05:00
3,Birmingham Highway Accident,Accident on the highway blocks traffic in Birmingham.,2024-10-12 08:15:00
17,Birmingham Highway Accident,The highway accident in Birmingham has been cleared.,2024-10-12 09:25:00
12,Hong Kong Political Reforms Protests,Protests in Hong Kong demanding political reforms.,2024-10-12 09:00:00
5,Industrial Complex Fire Incident,A large fire reported in an industrial complex.,2024-10-12 08:25:00
18,Industrial Complex Fire Incident,The industrial complex fire is spreading to nearby areas.,2024-10-12 09:30:00
7,Istanbul Earthquake,Earthquake detected near the city of Istanbul.,2024-10-12 08:35:00
20,Istanbul Earthquake,Aftershocks of the earthquake felt in Istanbul.,2024-10-12 09:40:00


event_name
Residential Area Fire Incident          3
London Tax Reform Protests              2
Birmingham Highway Accident             2
Paris Protests                          2
Industrial Complex Fire Incident        2
Madrid Protests                         2
Istanbul Earthquake                     2
New York Climate Policies Protest       2
Beirut Protest Explosion                2
Manchester Tax Reform Protests          1
Venice Flooding Incident                1
Hong Kong Political Reforms Protests    1
Berlin Car Accident Incident            1
Rome Protests                           1
Washington Climate Policies Protest     1
Name: count, dtype: int64

# Creating Flow Timeline

Overview of the Pipeline for Real-Time Event Management

To achieve your goal of managing, updating, and monitoring events in real-time, based on new data coming from Telegram, we can design a pipeline that works with your pre-defined list of event types (like protests, fires, etc.) and updates them as new messages come in. The pipeline will:

	1.	Define Event Types: Pre-set a list of known event types (like protests, fires, accidents).
	2.	Track Active Events: Maintain a list of currently active events, where each event has real-time updates.
	3.	Handle New Messages: For every new message, classify it into an existing event (if applicable), or create a new event if it introduces a new topic.
	4.	Update Events: Continuously update the information for each active event based on new messages.
	5.	Use GPT for Classification and Summarization: GPT will help to classify the incoming messages and enrich the data by providing event summaries and updates.

Pipeline Steps

	1.	Initialize Event Types and Active Events:
	•	Create a predefined list of event types, e.g., [“Protest”, “Fire”, “Accident”, etc.].
	•	Keep an event registry for tracking active events, which will get updated in real-time.
	2.	Process Incoming Telegram Messages:
	•	For each new message from Telegram:
	•	Use GPT to classify it based on event types.
	•	Check if the message belongs to an existing event (based on time, location, and content).
	•	If it matches an existing event, update that event with new information (using GPT to summarize the updates).
	•	If it doesn’t match, create a new event with a new entry in the active event registry.
	3.	Use GPT for Classification and Summarization:
	•	GPT will help by analyzing each incoming message, classifying it into an event type, and generating a concise summary of the event as it evolves.
	4.	Real-Time Updates:
	•	The system will run periodically (e.g., every minute) to check for new messages, classify them, and update events accordingly.

Proposed Code Structure

We can follow a structure similar to your example, where a TelegramMessageProcessor class handles incoming messages, and GPT is used for event classification and updates. Let’s start by defining our EventProcessor class, which will handle classification and updates in real-time.

# Main flow
client = OpenAIClient(api_key="your_openai_api_key")  # Assuming OpenAI client instance
processor = TelegramMessageProcessor(client)

# Process incoming message
new_message = "Massive protest in the city center about the new tax policy."
processor.process_new_message(new_message)

# Check active events
active_events = processor.get_active_events()
for event in active_events:
    print(f"Event Type: {event['type']}, Last Update: {event['last_update']}, Summary: {event['summary']}")

Key Components Explained:

	1.	Event Classification:
	•	GPT is used to classify the message into one of the predefined event types (Protest, Fire, Accident, Unknown), based on the content of the message.
	2.	Event Updates:
	•	Once the message is classified, the system checks if an event of that type is already active.
	•	If it is, it updates the event with the new message and regenerates the summary using GPT.
	•	If it’s a new event, the system creates a new event entry.
	3.	Summarization:
	•	GPT is used to summarize the series of messages related to an event. As more messages are received, the summary is updated.
	4.	Active Events Overview:
	•	The system provides a real-time view of all active events with their current summaries and timestamps for when they were last updated.

Real-Time Processing Strategy:

	•	The system can run periodically, say every minute, to check for new messages, classify them, and update events accordingly.
	•	The structure allows you to continuously enrich the information about an event as new details come in.

Example Scenario:

Let’s assume the following sequence of messages arrives:

	1.	Message 1: “Protest in London against tax reforms.”
	2.	Message 2: “Fire breaks out near the protest area in London.”
	3.	Message 3: “More protesters joining the march in Manchester.”

Message 1 will be classified as a “Protest” event, creating a new entry.
Message 2 might be classified as both “Protest” and “Fire”, depending on the system’s configuration, potentially updating both events or linking the events together.
Message 3 will update the “Protest” event with additional information.

As the messages are processed, the system will build a timeline of events with real-time updates and summaries.

This approach leverages GPT to handle both classification and summarization tasks while maintaining an up-to-date overview of active events.

In [40]:
from pydantic import BaseModel
from typing import List, Optional
from datetime import datetime

class EventUpdateResponse(BaseModel):
    """Response model for event updates."""
    event_type: str
    summary: str

class TelegramMessageProcessor:
    """Processor class to handle Telegram message classification and updates."""
    
    def __init__(self, client: OpenAIClient):
        self.client = client
        self.active_events = []  # List to track currently active events
    
    def classify_message(self, message: str) -> Optional[EventUpdateResponse]:
        """Classify a Telegram message into an event type and generate a summary."""
        system_message = """
            You are an event classifier. Classify the following message into one of the event types: Protest, Fire, Accident, or Unknown.

            After classification, summarize the message briefly. The summary should be clear and concise, capturing the key information of the event.

            Event Types:
            - Protest
            - Fire
            - Accident
            - Unknown
            
            Message:
        """
        
        response = self.client.chat(
            system_message=system_message,
            user_message=message,
            response_format=EventUpdateResponse
        )

        if response and response.choices and hasattr(response.choices[0].message, 'parsed'):
            return response.choices[0].message.parsed  # Parsed classification and summary
        return None
    
    def update_event(self, event_type: str, message: str):
        """Update an existing event with new information, or create a new event."""
        event_exists = False
        for event in self.active_events:
            if event['type'] == event_type:
                # Update existing event
                event['messages'].append(message)
                event['last_update'] = datetime.now()
                event['summary'] = self.summarize_event(event['messages'])
                event_exists = True
                break
        
        if not event_exists:
            # Create a new event
            new_event = {
                'type': event_type,
                'messages': [message],
                'first_report': datetime.now(),
                'last_update': datetime.now(),
                'summary': message  # Initial summary based on the first message
            }
            self.active_events.append(new_event)
    
    def summarize_event(self, messages: List[str]) -> str:
        """Summarize the event by combining related messages."""
        system_message = """
            Summarize the following set of messages into a concise and clear event summary:
        """
        message_list = "\n".join(messages)
        response = self.client.chat(
            system_message=system_message,
            user_message=message_list
        )
        
        return response.choices[0].message.content if response else "Summary unavailable."

    def process_new_message(self, message: str):
        """Process a new incoming Telegram message."""
        classification = self.classify_message(message)
        
        if classification:
            self.update_event(classification.event_type, message)
        else:
            print("Message could not be classified.")
    
    def get_active_events(self):
        """Get the current list of active events with summaries."""
        return [{"type": event['type'], "summary": event['summary'], "last_update": event['last_update']} for event in self.active_events]

# Main flow
import pandas as pd
from datetime import datetime, timedelta
import random

# Fake messages for demonstration purposes
fake_messages = [
    "Protest in London against tax reforms.",
    "Fire breaks out near the protest area in London.",
    "More protesters joining the march in Manchester.",
    "Accident on the highway blocks traffic in Birmingham.",
    "Massive protests erupt in downtown Paris.",
    "A large fire reported in an industrial complex.",
    "Protesters clash with police in Madrid.",
    "Earthquake detected near the city of Istanbul.",
    "More people join the protest in Barcelona.",
    "Firefighters are battling a fire in a residential area.",
    "Protest in New York over climate policies.",
    "Explosion heard near a protest in Beirut.",
    "Flooding reported after heavy rains in Venice.",
    "Protests in Hong Kong demanding political reforms.",
    "Accident involving multiple cars reported in Berlin."
]

# Generate random timestamps (simulating message times in the past 2 hours)
base_time = datetime.now()
timestamps = [base_time - timedelta(minutes=random.randint(1, 120)) for _ in range(len(fake_messages))]

# Create a pandas DataFrame to store messages and timestamps
df = pd.DataFrame({
    "message": fake_messages,
    "timestamp": timestamps
})

# Simulating the processing of the messages
client = OpenAIClient(show_prices=False)
processor = TelegramMessageProcessor(client)

# Process each fake message and store the results
for index, row in df.iterrows():
    message = row["message"]
    timestamp = row["timestamp"]
    print(f"Processing message at {timestamp}: {message}")
    
    processor.process_new_message(message)

# Check active events after processing all messages
active_events = processor.get_active_events()
for event in active_events:
    print(f"Event Type: {event['type']}, Last Update: {event['last_update']}, Summary: {event['summary']}")

# Display the DataFrame (with messages and timestamps)
df

Using closest match for encoding: 'gpt-4o' for input model 'gpt-4o-2024-08-06'
Resolved model for encoding: 'gpt-4o'
Using pricing model: 'gpt-4o-2024-08-06'
Processing message at 2024-10-14 00:06:25.862490: Protest in London against tax reforms.
Processing message at 2024-10-13 23:15:25.862490: Fire breaks out near the protest area in London.
Processing message at 2024-10-13 23:27:25.862490: More protesters joining the march in Manchester.
Processing message at 2024-10-13 23:09:25.862490: Accident on the highway blocks traffic in Birmingham.
Processing message at 2024-10-13 22:59:25.862490: Massive protests erupt in downtown Paris.
Processing message at 2024-10-13 23:02:25.862490: A large fire reported in an industrial complex.
Processing message at 2024-10-14 00:13:25.862490: Protesters clash with police in Madrid.
Processing message at 2024-10-13 23:45:25.862490: Earthquake detected near the city of Istanbul.
Processing message at 2024-10-13 23:20:25.862490: More people join the pro

,message,timestamp
0,Protest in London against tax reforms.,2024-10-14 00:06:25.862490
1,Fire breaks out near the protest area in London.,2024-10-13 23:15:25.862490
2,More protesters joining the march in Manchester.,2024-10-13 23:27:25.862490
3,Accident on the highway blocks traffic in Birmingham.,2024-10-13 23:09:25.862490
4,Massive protests erupt in downtown Paris.,2024-10-13 22:59:25.862490
5,A large fire reported in an industrial complex.,2024-10-13 23:02:25.862490
6,Protesters clash with police in Madrid.,2024-10-14 00:13:25.862490
7,Earthquake detected near the city of Istanbul.,2024-10-13 23:45:25.862490
8,More people join the protest in Barcelona.,2024-10-13 23:20:25.862490
9,Firefighters are battling a fire in a residential area.,2024-10-13 22:18:25.862490


In [42]:
active_events

[{'type': 'Protest',
  'summary': 'In late 2023, significant protests occurred globally in response to various political and social issues. London and Manchester witnessed demonstrations against tax reforms, while downtown Paris also saw considerable unrest. In Spain, protesters clashed with police in Madrid, with additional protests occurring in Barcelona. New York experienced demonstrations over climate policies. An explosion was reported near a protest in Beirut, and in Hong Kong, people protested demanding political reforms.',
  'last_update': datetime.datetime(2024, 10, 14, 0, 16, 0, 302282)},
 {'type': 'Fire',
  'summary': 'A fire occurred in an industrial complex near a protest area in London, with firefighters actively working to control the blaze.',
  'last_update': datetime.datetime(2024, 10, 14, 0, 15, 52, 132604)},
 {'type': 'Accident',
  'summary': 'There were two separate accidents reported: one blocking highway traffic in Birmingham and another involving multiple cars in